# Virtual inertia scheduling (VIS) for IEEE39

In [1]:
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import andes
import gurobipy as gb
import pandas as pd
import numpy as np
import logging
import torch
import os

logger = logging.getLogger(__name__)

from visopf import vis1, vis2, loadnn

## load data

In [2]:
# ----- get andes case from excel ------
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee39_vis.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

In [3]:
# ----- get neural network data ------
nn_path = '/VIS_opf/NN_train'
nn, norm = loadnn(nn_path)

## build optimization model and solve

In [4]:
# ----- set sim scenario -----
ss = vis2(norm=norm, nn=nn, dpe=0.1, rocof_lim=0.033, nadir_lim=0.01)

# ----- set vsg gen -----
vsg_ieee39 = ['PV_1', 'PV_6', 'PV_8', 'PV_9']
ss.from_andes(ssa, vsg_ieee39, Sbase=1000)

# ------ set gen cost ------
# iloc[0] - iloc[9]
# iloc[9]: slack bus
# iloc[0],[5],[8],[9]: vsg
ss.cost['c1'].iloc[0] = 1
ss.cost['c1'].iloc[5] = 0.1
ss.cost['c1'].iloc[9] = 1.5
ss.update_dict()

# ss.costdict

Restricted license - for non-production use only - expires 2023-10-25


In [5]:
ss.build()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.


In [6]:
ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,...,ramp30,type,p_pre,band,K,M,D,R,Mvsg,Dvsg
0,PV_1,1.0,PV_1,10.400,34.5,30,4.360864,15.0,2.0,1.035534,...,600,2,0,13.0,1,0.000000,0.0000,0.000000,10.4000,5.2000
1,PV_2,1.0,PV_2,8.360,34.5,31,6.460000,9.0,1.5,1.013246,...,600,1,0,7.5,1,6.738160,0.8360,0.059809,0.0000,0.0000
2,PV_3,1.0,PV_3,8.437,21.0,32,7.250000,8.0,1.0,1.020528,...,600,1,0,7.0,1,6.040892,0.8437,0.059263,0.0000,0.0000
3,PV_4,1.0,PV_4,11.748,21.0,33,6.520000,7.0,1.0,1.013430,...,600,1,0,6.0,1,10.244256,1.1748,0.042560,0.0000,0.0000
4,PV_5,1.0,PV_5,10.802,15.5,34,5.200000,7.0,1.0,1.019109,...,600,1,0,6.0,1,5.617040,1.0802,0.046288,0.0000,0.0000
5,PV_6,1.0,PV_6,10.857,15.5,35,6.900000,8.0,1.0,1.060000,...,600,2,0,7.0,1,0.000000,0.0000,0.000000,6.5142,5.4285
6,PV_7,1.0,PV_7,10.252,12.5,36,5.900000,7.0,1.0,1.060000,...,600,1,0,6.0,1,8.488656,1.0252,0.048771,0.0000,0.0000
7,PV_8,1.0,PV_8,9.702,12.5,37,3.300000,7.0,1.0,1.013996,...,600,2,0,6.0,1,0.000000,0.0000,0.000000,7.7616,2.9106
8,PV_9,1.0,PV_9,16.841,34.5,38,7.800000,10.0,1.0,1.052803,...,600,2,0,9.0,1,0.000000,0.0000,0.000000,10.1046,8.4205
9,Slack_10,1.0,Slack_10,11.990,345.0,39,5.741700,15.0,3.0,1.030000,...,600,1,0,12.0,1,11.990000,1.1990,0.041701,0.0000,0.0000


In [7]:
ss.get_res()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1040 rows, 678 columns and 11758 nonzeros
Model fingerprint: 0x19e88e41
Variable types: 358 continuous, 320 integer (320 binary)
Coefficient statistics:
  Matrix range     [2e-07, 1e+02]
  Objective range  [1e-01, 2e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [2e-03, 2e+03]
Found heuristic solution: objective 55.9204017
Presolve removed 556 rows and 340 columns
Presolve time: 0.02s
Presolved: 484 rows, 338 columns, 5669 nonzeros
Variable types: 182 continuous, 156 integer (156 binary)

Root relaxation: objective 5.286400e+01, 267 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dep

(        gen      Sn        pg       pru  prd
 0      PV_1  10.400  2.000000  0.007915  0.0
 1      PV_2   8.360  9.000000  0.000000  0.0
 2      PV_3   8.437  8.000000  0.000000  0.0
 3      PV_4  11.748  7.000000  0.000000  0.0
 4      PV_5  10.802  7.000000  0.000000  0.0
 5      PV_6  10.857  7.996833  0.003167  0.0
 6      PV_7  10.252  7.000000  0.000000  0.0
 7      PV_8   9.702  6.567167  0.007915  0.0
 8      PV_9  16.841  1.000000  0.007915  0.0
 9  Slack_10  11.990  3.000000  0.000000  0.0,
     gen     Mvsg      Dvsg    pg_vsg   pru_vsg  prd_vsg  pmax_vsg  pmin_vsg
 0  PV_1  0.29597  0.728619  2.000000  0.007915      0.0      15.0       2.0
 1  PV_6  0.00000  0.000000  7.996833  0.003167      0.0       8.0       1.0
 2  PV_8  0.29597  0.728619  6.567167  0.007915      0.0       7.0       1.0
 3  PV_9  0.29597  0.728619  1.000000  0.007915      0.0      10.0       1.0,
 {'Msys': 4.845483953459823,
  'Dsys': 0.835375317323098,
  'Rsys': 20.933442043222,
  'Fsys': 13.259510626